# HW5

For this homework, we are going to work with [*Indoor User Movement Prediction from RSS data*](https://archive.ics.uci.edu/ml/datasets/Indoor+User+Movement+Prediction+from+RSS+data) dataset from UCI.  The homework is due Friday, December 21st midnight. 

## Task 1

Download the dataset and unzip it in under a subdirectory of `data` folder named `rss_data`.

The files we are interested is in the subfolder `dataset`.  Each of these files whose names that start with `MovementAAL_RSS_` contain data collected from indivuduals. Each of these files represent a single data point.  There are 314 of these files, and hence, you have 314 data points.  Each file has 4 columns but the number of rows change from file to file.  

There is also a file named `MovementALL_target.csv` in that folder. This file tells us the class each of these measurements are assigned. Some of these measurements are labelled with +1 and some are labelled with -1.

## Task 2

Construct a SVM model that separates +1 labelled data points from -1 data points.  You must first solve the problem that these datapoints do not have the same number of rows even though they all have the same number of columns. 

## Task 3

Using [Keras](https://keras.io/getting-started/sequential-model-guide/) write a neural network model that separates +1 labelled data points from -1 data points.

## Notes

1. You must document each step of your tasks: what are you doing, why are you doing it, what problems you encountered and how you solved it.  All of these must be explained and documented.  Solutions without sufficient documentations will be penalized accordingly. 50% of your points will come from your code, while the other 50% will come from your explanations.

1. You can use MS Excel to inspect the files, but loading them up to python using pandas and inspecting them there under jupyter is easier.

3. Put the data in a separate subfolder of your `data` folder and rename it `rss_data`. I'll take points off if the data is not saved under the correct place.

1. For both of Task 2 and Task 3, you must split your data into a train and test set, and then evaluate the accuracy of your model on the test set.



## Task 1 : SVM

Veri kümesini indirdim ve sonrasında inceledim. -1 yazıyorsa değiştirmedi demek 1 diyorsa değiştirdi demek. Sonrasında bu kümeyi test ve train kümelerine ayırmam gerekti. Tabi ki başlangıçta nasıl ayıracağımı bulamadım. Sonrasında danıştığım insanlar sayesinde aşağıdaki yolu buldum. Öncelikle import edilmesi gereken şeyleri edelim.



In [1]:
import random
from numpy import *
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Şimdi de verileri ekleyelim.Birden fazla verimiz olduğu için hepsini tek tek almıycaz bunun için bir fonksiyon oluşturup onunla hepsini birden alacağız. 

In [2]:
ds_target = pd.read_csv('../data/rss_data/dataset/MovementAAL_target.csv',
                                   names=('sequence_ID','class_label'),
                                   skiprows=(1)) # ds = dataset ve skiprows diyoruz çünkü ilk satır yazı biz sayıları alıyoruz.

In [3]:
def get_dataset(identity):
    return pd.read_csv('../data/rss_data/dataset/MovementAAL_RSS_%s.csv' %identity,
                                  names=('RSS_anchor1','RSS_anchor2','RSS_anchor3','RSS_anchor4'),
                                  skiprows=(1))

Aldığımız verileri birleştirmemiz gerekiyor çünkü data setimiz parça parça şeklinde. Onları vektör gibi düşünüp birleştirceğiz. Uzunlukları eşit olmadığı için de ayrıcadan işlem yapacağız. Oluşturduktan sonra test ve train şeklinde bölmeye başlayabiliriz.

In [4]:
def create(ds):
    sets  = []
    target = []
    for identity_x, row in ds.iterrows():
        data = get_dataset(row['sequence_ID'])
        sets.append(data)
        array = [row['class_label']]*len(data)
        target += array 
        
    return pd.concat(sets), target

data_test, target_test = create(ds_target)

Boş olan dataları 0 ile doldurmuş olduk.

In [5]:
x = data_test
y = target_test


x_train = x[ : int(random.uniform(0,13196))]
y_train = y[ : int(random.uniform(1319,13197))]
x_test = x[ :  int(random.uniform(0,13196))]
y_test = y[ : int(random.uniform(13197,13197))]

Burada ayırdığımız veriyi aşağıdada da kullanmamız gerekecek Keras için de lazım

In [6]:
classification =SVC(kernel = 'rbf' , gamma = 5.0)

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25)

classification.fit(x_train, y_train)
predicted = classification.predict(x_test)

print(confusion_matrix(y_test, predicted))
accuracy_score(y_test,predicted)

[[ 841  538]
 [ 335 1586]]


0.7354545454545455

c (gamma ya da aldığımız kernela bağlı olan parametre) değişince değişmiyor ancak kernel değişince değişiyor.

## Task 2 : Keras

In [7]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense , LSTM , Dropout , Conv1D
from keras.optimizers import Adamax , RMSprop , SGD 
import matplotlib.pyplot as plt

C:\Users\adile\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Yukardakiler Keras kütüphanesini çalıştırabilmemiz için import etmemiz gerekenler.

In [8]:
K.clear_session()

model = Sequential()

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25)


model.add(Dense(100, input_shape = (4,), activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(30, activation = 'relu'))
model.add(Dropout(0.01))
model.add(Dense(1, activation = 'relu'))


model.compile(optimizer=Adamax(lr=0.004), loss='mean_absolute_error')
model.fit(np.array(x_train), np.array(y_train) , batch_size=20 , epochs=10)

Epoch 1/10
9897/9897 [==============================] - 1s 92us/step - loss: 0.8106
Epoch 2/10
9897/9897 [==============================] - 1s 61us/step - loss: 0.7657
Epoch 3/10
9897/9897 [==============================] - 1s 57us/step - loss: 0.7542
Epoch 4/10
9897/9897 [==============================] - 1s 59us/step - loss: 0.7494
Epoch 5/10
9897/9897 [==============================] - 1s 56us/step - loss: 0.7457
Epoch 6/10
9897/9897 [==============================] - 1s 62us/step - loss: 0.7413
Epoch 7/10
9897/9897 [==============================] - 1s 64us/step - loss: 0.7388
Epoch 8/10
9897/9897 [==============================] - 1s 60us/step - loss: 0.7354
Epoch 9/10
9897/9897 [==============================] - 1s 60us/step - loss: 0.7324
Epoch 10/10
9897/9897 [==============================] - 1s 61us/step - loss: 0.7313


Sürekli artmasını beklerdim ancak azalması beni şaşırttı demek ki bu veri kümesinde bu hatayı veriyor.